<a href="https://colab.research.google.com/github/lekshmi-raj/KTU-MCA---SGPA-AND-GRADE-PREDICTION/blob/main/flask_univ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## RABBET SHAPE POLYGON LOGIN FOR ADMIN & USER
## SLANTIN MENU FROM LEFT TOP TO BOTTOM IS IN THE HOME PAGE
########
## Separate Sign Up Options provided for Admin & User, Newly added

########

# Scroll this cell automatically to  Result grid

from IPython.display import display, HTML, Javascript
def set_focus_cell1():
    display(Javascript("""
        document.getElementById('output-area').scrollIntoView({behavior: 'smooth'});
    """))

# Call set_focus_cell1 to scroll down automatically this cell

set_focus_cell1()

# Install necessary packages

!pip install pyngrok

## Import pandas, flask & pyngrok libraries

import numpy as np
import argparse
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Input

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
# Dataset Manipulation
import os
import pandas as pd

## joblib is used to save the one hot encoder used during the training Process
## this is needed while predicting actual diabetes output using the saved model

import joblib

## Import Flask and ngrok libraries
from flask import Flask, render_template_string, request, redirect, url_for, session, flash, send_from_directory
from flask import send_file

from pyngrok import ngrok
import threading
import os
import csv
import time

# ## Import PDF converted packages
# ## Install this to convert your HTML marklist to PDF

pdf_install = """
<!DOCTYPE html>
<html>
<head>
  <title>Installing PDF Tool</title>
  <style>
    body {
      background-color: #f5f5f5;
      font-family: Arial, sans-serif;
      text-align: left;
      padding-top: 50px;
    }
    h2 {
      color: #2c3e50;
    }
  </style>
</head>
<body>
  <h3>Please Wait !!!</h3>
</body>
</html>
"""
display(HTML(pdf_install))

!pip install weasyprint > /dev/null 2>&1
import weasyprint

# Kill any process running on port 5000
# Otherwise you will have to 'Restart colab Session' after 3 continuous
# run of this code cell

ngrok.kill()  # Kill any active ngrok processes

# Authenticate with ngrok (use your authtoken here)
# Goto ngrok.com-->Click Login -->Continue with Google-->
# Your Authentication[From Left panel]-->Copy the Authtoken

ngrok.set_auth_token("2wAEJwIV0oqWvJm3BKwivwOniec_4htrY6MgV5sHaquD1Voie")

# Initialize Flask app and set a secret key for session management

app = Flask(__name__)
app.secret_key = 'your_secret_key'  # Required for session management and flash messages

# Serve image from '/content/drive/MyDrive/' folder in Google Drive

@app.route('/content/drive/MyDrive/<filename>')
def custom_static(filename):
    return send_from_directory('/content/drive/MyDrive', filename)


# Login HTML Template

LOGIN_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Login</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-image: url("/content/drive/MyDrive/kvm_img3.jpg"); /* Replace with your real ID */
            background-size: cover;
            background-repeat: no-repeat;
            background-position:center;
            text-align: center;
            padding-top: 100px;
            height: 100vh;
        }
        .container {
            display: flex;
            justify-content: center;
            gap: 50px;
        }
        .login-form {
          background-color: white;
          padding: 20px;
          border-radius: 0px; /* Optional: Set to 10px if you want rounded rectangle */
          box-shadow: 2 2px 10px rgba(0, 0, 0, 0.1);
          width: 300px;
          margin-top: 70px;

          /* Removed clip-path to make it a regular rectangle */
      }


        .login-form input {
            display: block;
            margin: 10px auto;
            padding: 10px;
            width: 80%;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        .login-form button {
            padding: 10px 20px;
            background-color:#001f3f ;
            color: white;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            width: 85%;
        }
        .login-form button:hover {
            background-color: #001f3f;
        }
        .error {
            color: red;
            margin-bottom: 15px;
        }
        h1 {
            color: white;
            font-weight: bold;
        }
    </style>
</head>

<body>
    <h1>Welcome to the KTU-MCA Score Prediction</h1>

    <div class="container">
<!-- Admin Login Form -->

    <div class="login-form">
        <h3>Admin Login</h3>
        {% if admin_error %}
        <p class="error">{{ admin_error }}</p>
        {% endif %}
        <form action="/admin-login" method="POST">
            <input type="text" name="username" placeholder="Admin ID" required \
            autocomplete="off" readonly onfocus="this.removeAttribute('readonly');">
            <input type="password" name="password" placeholder="Password" required>
            <button type="submit">Login</button>
        </form>

    </div>

<!-- User Login Form -->

    <div class="login-form">
        <h3>User Login</h3>
        {% if user_error %}
        <p class="error">{{ user_error }}</p>
        {% endif %}
        <form action="/user-login" method="POST">
            <input type="text" name="username" placeholder="User ID" required>
            <input type="password" name="password" placeholder="Password" required>
            <button type="submit">Login</button>
        </form>
<!-- SignUp option -->
        <p>Don't have an Account? <a href="#" id="user-signup-link">Sign Up</a></p>
    </div>

<!-- Hidden Signup Form USER-->

    <div class="user-signup-form" style="display: none;">
        <h3>USER Sign Up</h3>
        <form action="/user-signup" method="POST">
            <input type="text" name="signup-username" placeholder="USER ID" required>
            <input type="password" name="signup-password" placeholder="Password" required>
            <button type="submit">Save</button>
        </form>
    </div>
<!-- JavaScript to Toggle Signup Form Visibility USER-->
    <script>
        document.getElementById('user-signup-link').addEventListener('click', function(event) {
            event.preventDefault();
            const signupForm = document.querySelector('.user-signup-form');

// Toggle the signup form's visibility
            if (signupForm.style.display === 'block') {
                signupForm.style.display = 'none';  // Hide the signup form
            } else {
                signupForm.style.display = 'block';  // Show the signup form
            }
        });
    </script>

    </div>
</body>
</html>
"""

# Route for handling admin signup and writing to the CSV file

@app.route("/admin-signup", methods=["POST"])

def admin_signup():
    admin_id = request.form.get("signup-username")
    password = request.form.get("signup-password")

# Path to the CSV file
    csv_file_path = '/content/drive/MyDrive/Admin_KTU.csv'

# Check if the file exists or is empty, and write headers if necessary
    file_exists = os.path.isfile(csv_file_path) and os.path.getsize(csv_file_path) > 0

    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)

# Write the headers if the file is new or empty
        if not file_exists:
            writer.writerow(['Admin_ID', 'Password'])

# Write the new admin credentials
        writer.writerow([admin_id, password])

# Render the login form after signup
    return render_template_string(LOGIN_TEMPLATE)

# Route for handling USER signup and writing to the CSV file

@app.route("/user-signup", methods=["POST"])

def user_signup():
    user_id = request.form.get("signup-username")
    password = request.form.get("signup-password")

# Path to the CSV file
    csv_file_path = '/content/drive/MyDrive/User_KTU.csv'

# Check if the file exists or is empty, and write headers if necessary
    file_exists = os.path.isfile(csv_file_path) and os.path.getsize(csv_file_path) > 0

    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)

# Write the headers if the file is new or empty
        if not file_exists:
            writer.writerow(['User_ID', 'Password'])

# Write the new user credentials
        writer.writerow([user_id, password])

# Render the login form after signup
    return render_template_string(LOGIN_TEMPLATE)

# HTML Template for the main page (from your previous implementation)


HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>KTU SGPA Prediction</title>
    <style>
    /* Fullscreen background image CSS */
            /* Fullscreen background image CSS */
      body{
      background-image: url("/content/drive/MyDrive/kvm_img1.jpg");
      background-size:cover; /* Keeps all images' aspect ratios */
      background-position:center; /* Positions images */
      background-repeat: no-repeat;
      height: 300px;
      margin: 0;
      width: 100vw;
    }

        }
      marquee {
          color: white;
          font-size: 34px;
          font-weight: bold;
          background: rgba(0, 0, 0, 0.5);
          padding: 10px;
          margin-top: 200px; /* Pushes it down below the background image */
          display: block;
          }
        /* Optional: Add some content */
        .content {
          color: white;
          text-align: center;
          position: absolute;
          top: 25%;
          left: 50%;
          transform: translate(-50%, -50%);
          font-size: 2rem;
          font-family: Arial, sans-serif;
        }
        .abstract {
            margin: 20px auto;
            max-width: 900px;
            padding: 15px;
            background: #FFFFE0; /* Light yellow background */
            color: #155724;
            font-size: 18px;
            font-weight: bold; /* Makes the text bold */
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
            border-radius: 5px;
            border: 1px solid #c3e6cb;
        }
        .buttons {
            margin-top: 300px;
            display: flex;
            flex-direction: column;
            align-items: flex-start; /* Align buttons to the left */
            gap: 10px;


            background-image: url('/content/drive/MyDrive/login_background_1.jpg');
            background-size: cover;
            background-position: center;
            padding: 30px;

            border-radius: 0px; /* Make it a sharp-cornered rectangle */
            margin-left: 10px;
            color: white;
            box-shadow: 0 4px 10px rgba(0, 0, 0, 0.3);

            width: 300px; /* Set an explicit width */
            height: 400px; /* Already defined */
        }



        button_1 {
            min-width: 300px;
            margin: 10px;
            padding: 10px 20px;
            font-size: 18px;
            font-weight: bold;
            color: white;
            background: linear-gradient(to right, green, blue);  /* Background color for the buttons */
            border: none;
            border-radius: 5px;
            cursor: pointer;
            width: auto;  /* Ensures the button width only fits its content */
            margin-left: 120px;
            text-align: center;  /* Centers text inside the button */
            display: inline-block;  /* Ensures the button behaves like a block to center text */
        }

        button_1:hover {
            background: linear-gradient(to right, white, grey, grey);
        }

        button_2 {
            min-width: 300px;
            margin: 10px;
            padding: 10px 20px;
            font-size: 18px;
            font-weight: bold;
            color: white;
            background: linear-gradient(to right, green, blue);  /* Background color for the buttons */
            border: none;
            border-radius: 5px;
            cursor: pointer;
            margin-left: 220px; /* Space from the left */
            width: auto;  /* Ensures the button width only fits its content */
            text-align: center;  /* Centers text inside the button */
            display: inline-block;  /* Ensures the button behaves like a block to center text */
        }

        button_2:hover {
            background: linear-gradient(to right, white, grey, grey);
        }

        button_3 {
            min-width: 300px;
            margin: 10px;
            padding: 10px 20px;
            font-size: 18px;
            font-weight: bold;
            color: white;
            background: linear-gradient(to right, green, blue);  /* Background color for the buttons */
            border: none;
            border-radius: 5px;
            cursor: pointer;
            margin-left: 330px; /* Space from the left */
            width: auto;  /* Ensures the button width only fits its content */
            text-align: center;  /* Centers text inside the button */
            display: inline-block;  /* Ensures the button behaves like a block to center text */

        }

        button_3:hover {
            background: linear-gradient(to right, white, grey, grey);
        }

        button_4 {
            min-width: 300px;
            margin: 10px;
            padding: 10px 20px;
            font-size: 18px;
            font-weight: bold;
            color: white;
            background: linear-gradient(to right, green,blue);  /* Background color for the buttons */
            border: none;
            border-radius: 5px;
            cursor: pointer;
            margin-left: 430px; /* Space from the left */
            width: auto;  /* Ensures the button width only fits its content */
            text-align: center;  /* Centers text inside the button */
            display: inline-block;  /* Ensures the button behaves like a block to center text */

        }

        button_4:hover {
            background: linear-gradient(to right, white, grey, grey);
        }

        button_5 {
            min-width: 300px;
            margin: 10px;
            padding: 10px 20px;
            font-size: 18px;
            font-weight: bold;
            color: white;
            background: linear-gradient(to right, green, blue);  /* Background color for the buttons */
            border: none;
            border-radius: 5px;
            cursor: pointer;
            margin-left: 530px; /* Space from the left */
            width: auto;  /* Ensures the button width only fits its content */
            text-align: center;  /* Centers text inside the button */
            display: inline-block;  /* Ensures the button behaves like a block to center text */

        }

        button_5:hover {
            background: linear-gradient(to right, white, gray, gray);
        }

        .text_right_side {
            max-width: 400px;
            padding: 10px;
            font-size: 20px;
            font-weight: bold;
            color: blue;
            margin-top: 300px;  /* Moves the text 500px down */
            text-align: center;       /* Centers text horizontally */
        }


        .accuracy {
            margin: 20px auto;
            max-width: 600px;
            padding: 15px;
            background: #d4edda;
            color: #155724;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
            border-radius: 5px;
            border: 1px solid #c3e6cb;
            display: none; /* Hide the form initially */
        }
        accuracy:hover {
            background-color: #a71d2a;
        }

        .user-details {
            margin: 20px auto;
            max-width: 800px;
            padding: 15px;
            background: #d4edda;
            color: #155724;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
            border-radius: 5px;
            border: 1px solid #c3e6cb;
            display: none; /* Hide the form initially */
        }
        form {
            text-align: left;
        }
        form .input-group {
            margin: 10px 0;
            display: flex;
            align-items: center;
            justify-content: space-between;
        }
        form input, form select {
            padding: 10px;
            width: 70%;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        form label {
            font-weight: bold;
            width: 30%;
            margin-right: 10px;
        }
        #wait-message {
            display: none;
            font-size: 25px;
            color: red;
            font-weight: bold;
            text-align: left; /* Align message to the left */
            margin-left: 20px; /* Optional: Adds some space on the left */
            margin-bottom: 20px; /* Adds vertical space below the message */
        }

    </style>
    <script>
        function toggleUserDetails() {
            var userDetails = document.getElementById('user-details-form');

            if (userDetails.style.display === 'none' || userDetails.style.display === '') {
                userDetails.style.display = 'block';

// Scroll into view
                userDetails.scrollIntoView({ behavior: 'smooth' });

// Focus on first input after scroll
                setTimeout(function () {
                    document.getElementById('regno').focus();
                }, 500);
            } else {
                userDetails.style.display = 'none';
            }
        }

        function showWaitMessage() {
            const waitMessage = document.getElementById("wait-message");
            waitMessage.style.display = "block";
 // Scroll to the element smoothly
            waitMessage.scrollIntoView({ behavior: 'smooth' });
        }
    </script>

</head>

    <header>
       <h1 style="color: white;text-align: center;">KTU-MCA Score Prediction</h1>
       <h2 style="color: white;text-align: center;">"Learning gives creativity, creativity leads to thinking, thinking provides knowledge, and knowledge makes you great"</h2>
    </header>
    <main>
    <div class="container">
        <div class="buttons">
            <button_1 onclick="location.href='/AboutThis';">About This Project</button_1>

            <button_2 onclick="toggleUserDetails(); " title="Use this for Existing Subscriber"> \
            Input Subjects & Grades</button_2>

            <button_3 onclick="showWaitMessage(); location.href='/predict';">Predictions</button_3>
            <button_4 onclick="location.href='/accuracyDetails';">Accuracy</button_4>

            <button_5 onclick="checkAccess('{{ role }}')" title="To Add new Subscriber, \
            click the Button and run your Program in COLAB..">View & Run Program Code</button_5>

            <p></p>
            <p></p>

        </div>

<script>
  function checkAccess(role) {
      if (role.trim() === 'Admin') {
// If the user is an admin, allow access to the code link $$
          window.location.href = 'https://colab.research.google.com/drive/14Z-zwm3TzBkvurKOo5ephs8Vrmfwze6X';
      } else {
          // For non-admin users, show 'Access Denied' and redirect to home
          alert('This Option is Only for Admin ... Access Denied !!!');
          window.location.href = '/';
      }
  }
</script>
        <div>
        <p id="wait-message">Loading !!!</p>
        <br>
        </div>

        <section id="user-details-form" class="user-details">

        <h2 style="color: red;">Help :- SEM1, SEM2, SEM3 - Select 4 Credit Subjs. first then 2 Credits</h2>
        <h2 style="color: red;">&gt;&gt;&gt;&gt;&gt;:- SEM4 - Select 6 Credit, 2 Credit, 12 Credit Subjs. first</h2>

            <form action="/save-details" method="post">

                <div class="input-group">
                    <label for="regno">Registration Number</label>
                    <input type="text" id="regno" name="regno">
                </div>

                <div class="input-group">
                    <label for="sname">Student Name</label>
                    <input type="text" id="sname" name="sname">
                </div>

                <div class="input-group">
                    <label for="semester">Semester</label>
                    <select id="semester" name="semester">
                        <option value="1">1</option>
                        <option value="2">2</option>
                        <option value="3">3</option>
                        <option value="4">4</option>
                    </select>
                </div>

<!-- Subject Group subj1,
wrapper is used to hide subj4 onwards when user select
Semester = 4, the corresponding Javascrip is under <script> Tag below
-->

                    <div id="subj1-wrapper" style="display: flex; align-items: center; gap: 25px; margin-bottom: 15px;">
                        <div style="display: flex; align-items: center; gap: 8px;">
                            <label for="subj1" style="min-width: 80px;">Subject 1</label>
                              <select name="subj1" id="subj1" style="min-width: 400px;">
                                      {% for subj, credit in subj_name_options %}
                                          {% set name = subj.split(':::')[0] %}
                                          <option value="{{ subj }}" {% if loop.first %}selected{% endif %}>
                                              {{ name }} ({{ credit }} credits)
                                          </option>
                                      {% endfor %}
                               </select>
                        </div>

<!-- Grade Group subj1-->
                        <div style="display: flex; align-items: center; gap: 8px; margin-left: 50px;">
                            <label for="grade1" style="min-width: 60px;">Grade</label>
                            <select name="grade1" id="grade1" style="min-width: 100px;">
                                <option value="S" selected>S</option>
                                <option value="A+">A+</option>
                                <option value="A">A</option>
                                <option value="B+">B+</option>
                                <option value="B">B</option>
                                <option value="C+">C+</option>
                                <option value="C">C</option>
                                <option value="D">D</option>
                                <option value="P">P</option>
                                <option value="F">F</option>
                            </select>
                        </div>
                     </div>

<!-- Subject Group subj2-->

                    <div id="subj2-wrapper" style="display: flex; align-items: center; gap: 25px; margin-bottom: 15px;">
                        <div style="display: flex; align-items: center; gap: 8px;">
                            <label for="subj2" style="min-width: 80px;">Subject 2</label>
                              <select name="subj2" id="subj2" style="min-width: 400px;">
                                      {% for subj, credit in subj_name_options %}
                                          {% set name = subj.split(':::')[0] %}
                                          <option value="{{ subj }}" {% if loop.first %}selected{% endif %}>
                                              {{ name }} ({{ credit }} credits)
                                          </option>
                                      {% endfor %}
                               </select>
                        </div>

<!-- Grade Group subj2-->
                        <div style="display: flex; align-items: center; gap: 8px; margin-left: 50px;">
                            <label for="grade2" style="min-width: 60px;">Grade</label>
                            <select name="grade2" id="grade2" style="min-width: 100px;">
                                <option value="S" selected>S</option>
                                <option value="A+">A+</option>
                                <option value="A">A</option>
                                <option value="B+">B+</option>
                                <option value="B">B</option>
                                <option value="C+">C+</option>
                                <option value="C">C</option>
                                <option value="D">D</option>
                                <option value="P">P</option>
                                <option value="F">F</option>
                            </select>
                        </div>
                    </div>

<!-- Subject Group subj3-->

                    <div id="subj3-wrapper" style="display: flex; align-items: center; gap: 25px; margin-bottom: 15px;">
                        <div style="display: flex; align-items: center; gap: 8px;">
                            <label for="subj3" style="min-width: 80px;">Subject 3</label>
                              <select name="subj3" id="subj3" style="min-width: 400px;">
                                      {% for subj, credit in subj_name_options %}
                                          {% set name = subj.split(':::')[0] %}
                                          <option value="{{ subj }}" {% if loop.first %}selected{% endif %}>
                                              {{ name }} ({{ credit }} credits)
                                          </option>
                                      {% endfor %}
                               </select>
                        </div>

<!-- Grade Group subj3-->
                        <div style="display: flex; align-items: center; gap: 8px; margin-left: 50px;">
                            <label for="grade3" style="min-width: 60px;">Grade</label>
                            <select name="grade3" id="grade3" style="min-width: 100px;">
                                <option value="S" selected>S</option>
                                <option value="A+">A+</option>
                                <option value="A">A</option>
                                <option value="B+">B+</option>
                                <option value="B">B</option>
                                <option value="C+">C+</option>
                                <option value="C">C</option>
                                <option value="D">D</option>
                                <option value="P">P</option>
                                <option value="F">F</option>
                            </select>
                        </div>
                    </div>

<!-- Subject Group subj4-->

                    <div id="subj4-wrapper" style="display: flex; align-items: center; gap: 25px; margin-bottom: 15px;">
                        <div style="display: flex; align-items: center; gap: 8px;">
                            <label for="subj4" style="min-width: 80px;">Subject 4</label>
                              <select name="subj4" id="subj4" style="min-width: 400px;">
                                      {% for subj, credit in subj_name_options %}
                                          {% set name = subj.split(':::')[0] %}
                                          <option value="{{ subj }}" {% if loop.first %}selected{% endif %}>
                                              {{ name }} ({{ credit }} credits)
                                          </option>
                                      {% endfor %}
                               </select>
                        </div>

<!-- Grade Group subj4-->
                        <div style="display: flex; align-items: center; gap: 8px; margin-left: 50px;">
                            <label for="grade4" style="min-width: 60px;">Grade</label>
                            <select name="grade4" id="grade4" style="min-width: 100px;">
                                <option value="S" selected>S</option>
                                <option value="A+">A+</option>
                                <option value="A">A</option>
                                <option value="B+">B+</option>
                                <option value="B">B</option>
                                <option value="C+">C+</option>
                                <option value="C">C</option>
                                <option value="D">D</option>
                                <option value="P">P</option>
                                <option value="F">F</option>
                            </select>
                        </div>
                    </div>

<!-- Subject Group subj5-->

                    <div id="subj5-wrapper" style="display: flex; align-items: center; gap: 25px; margin-bottom: 15px;">
                        <div style="display: flex; align-items: center; gap: 8px;">
                            <label for="subj5" style="min-width: 80px;">Subject 5</label>
                              <select name="subj5" id="subj5" style="min-width: 400px;">
                                      {% for subj, credit in subj_name_options %}
                                          {% set name = subj.split(':::')[0] %}
                                          <option value="{{ subj }}" {% if loop.first %}selected{% endif %}>
                                              {{ name }} ({{ credit }} credits)
                                          </option>
                                      {% endfor %}
                               </select>
                        </div>

<!-- Grade Group subj5-->
                        <div style="display: flex; align-items: center; gap: 8px; margin-left: 50px;">
                            <label for="grade5" style="min-width: 60px;">Grade</label>
                            <select name="grade5" id="grade5" style="min-width: 100px;">
                                <option value="S" selected>S</option>
                                <option value="A+">A+</option>
                                <option value="A">A</option>
                                <option value="B+">B+</option>
                                <option value="B">B</option>
                                <option value="C+">C+</option>
                                <option value="C">C</option>
                                <option value="D">D</option>
                                <option value="P">P</option>
                                <option value="F">F</option>
                            </select>
                        </div>
                    </div>

<!-- Subject Group subj6-->

                    <div id="subj6-wrapper" style="display: flex; align-items: center; gap: 25px; margin-bottom: 15px;">
                        <div style="display: flex; align-items: center; gap: 8px;">
                            <label for="subj6" style="min-width: 80px;">Subject 6</label>
                              <select name="subj6" id="subj6" style="min-width: 400px;">
                                      {% for subj, credit in subj_name_options %}
                                          {% set name = subj.split(':::')[0] %}
                                          <option value="{{ subj }}" {% if loop.first %}selected{% endif %}>
                                              {{ name }} ({{ credit }} credits)
                                          </option>
                                      {% endfor %}
                               </select>
                        </div>

<!-- Grade Group subj6-->
                        <div style="display: flex; align-items: center; gap: 8px; margin-left: 50px;">
                            <label for="grade6" style="min-width: 60px;">Grade</label>
                            <select name="grade6" id="grade6" style="min-width: 100px;">
                                <option value="S" selected>S</option>
                                <option value="A+">A+</option>
                                <option value="A">A</option>
                                <option value="B+">B+</option>
                                <option value="B">B</option>
                                <option value="C+">C+</option>
                                <option value="C">C</option>
                                <option value="D">D</option>
                                <option value="P">P</option>
                                <option value="F">F</option>
                            </select>
                        </div>
                    </div>

<!-- Subject Group subj7-->

                    <div id="subj7-wrapper" style="display: flex; align-items: center; gap: 25px; margin-bottom: 15px;">
                        <div style="display: flex; align-items: center; gap: 8px;">
                            <label for="subj7" style="min-width: 80px;">Subject 7</label>
                              <select name="subj7" id="subj7" style="min-width: 400px;">
                                      {% for subj, credit in subj_name_options %}
                                          {% set name = subj.split(':::')[0] %}
                                          <option value="{{ subj }}" {% if loop.first %}selected{% endif %}>
                                              {{ name }} ({{ credit }} credits)
                                          </option>
                                      {% endfor %}
                               </select>
                        </div>

<!-- Grade Group subj7-->
                        <div style="display: flex; align-items: center; gap: 8px; margin-left: 50px;">
                            <label for="grade7" style="min-width: 60px;">Grade</label>
                            <select name="grade7" id="grade7" style="min-width: 100px;">
                                <option value="S" selected>S</option>
                                <option value="A+">A+</option>
                                <option value="A">A</option>
                                <option value="B+">B+</option>
                                <option value="B">B</option>
                                <option value="C+">C+</option>
                                <option value="C">C</option>
                                <option value="D">D</option>
                                <option value="P">P</option>
                                <option value="F">F</option>
                            </select>
                        </div>
                    </div>

                <button type="submit" style="background-color: blue; color: white; width: 120px; height: 40px;cursor: pointer;">
                 S a v e
                </button>
<!-- JS to control visibility when user entered Semester = 4 -->
                <script>
                    function handleSemesterChange() {
                        const semester = document.getElementById("semester").value;
                        const totalSubjects = 7;

                        for (let i = 1; i <= totalSubjects; i++) {
                            const wrapper = document.getElementById(`subj${i}-wrapper`);
                            if (!wrapper) continue;

                            if (semester === "4" && i > 3) {
                                wrapper.style.display = "none";
                            } else {
                                wrapper.style.display = "flex";
                            }
                        }
                    }

                    document.addEventListener("DOMContentLoaded", function () {
                        const semesterDropdown = document.getElementById("semester");
                        handleSemesterChange(); // Call once initially
                        semesterDropdown.addEventListener("change", handleSemesterChange);
                    });
                </script>

            </form>
        </section>
    </main>
       <marquee behavior="scroll" direction="left">
        Designed and Developed by ... LEKSHMI RAJ G
    </marquee>
</body>
</html>
"""

# Flask route to render the main page

@app.route('/', methods=['GET', 'POST'])
def home():
    # Check if the user is logged in
    if not session.get('logged_in'):
        return redirect(url_for('admin_login'))  # Redirect to login if not logged in

# Get the user's role from the session

    role = str(session.get('role', 'User')) # Default to 'User' if no role is set

    selected_subj_name = None
#    selected_airport_name = None

    if request.method == 'POST':
        selected_subj_name_options = request.form['subj_name']

# Render the template with flight Feature values $$

# Render the template with flight Feature values
    return render_template_string(
        HTML_TEMPLATE,
        subj_name=selected_subj_name,
        selected_subj_name=selected_subj_name,
        role=role,
        subj_name_options=subj_name_options
    )

# Admin login route
@app.route('/admin-login', methods=['GET', 'POST'])
def admin_login():
    csv_file_path = '/content/drive/MyDrive/Admin_KTU.csv'

    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        # Read CSV and check if the username and password match
        with open(csv_file_path, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row['Admin_ID'] == username and row['Password'] == password:
                    session['logged_in'] = True
                    session['role'] = 'Admin'  # Set session role as Admin
                    return redirect(url_for('home'))

        # If no match is found
        return "<br><h2>Admin Login Error..Re-enter...Go Back to.. <a href='/admin-login'>Main Page..</a>.</h2>"

    return render_template_string(LOGIN_TEMPLATE)


# User login route
@app.route('/user-login', methods=['GET', 'POST'])
def user_login():
    csv_file_path = '/content/drive/MyDrive/User_KTU.csv'

    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        # Read CSV and check if the username and password match
        with open(csv_file_path, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row['User_ID'] == username and row['Password'] == password:
                    session['logged_in'] = True
                    session['role'] = 'User'  # Set session role as User
                    return redirect(url_for('home'))

        # If no match is found
        return "<br><h2>USER Login Error..Re-enter...Go Back to.. <a href='/'>Main Page..</a>.</h2>"

    return render_template_string(LOGIN_TEMPLATE)

# Flask route to save user details as CSV (from your previous implementation)

# Read the course_name_and_credits CSV file
df_course_names = pd.read_csv('/content/drive/MyDrive/course_name_and_credits.csv')
subject_names = sorted(df_course_names['semester'].dropna().unique().tolist())

# Extract the column containing course names, for dropdown box

subj_name_options = [
    (f"{row.course_name}:::{row.credits}", row.credits)
    for row in df_course_names.itertuples(index=False)
]

@app.route('/save-details', methods=['POST'])
def save_details():

# File path to save the data in Google Drive (CSV format)

    file_path = '/content/drive/MyDrive/Single_Univ_Data.csv'

    headers = ["Regn_No", "Name","Semester","subj1","subj2","subj3","subj4","subj5",
             "subj6","subj7", "subj1_grade", "subj2_grade", "subj3_grade", "subj4_grade",
             "subj5_grade", "subj6_grade", "subj7_grade","subj1_credits", "subj2_credits", "subj3_credits",
             "subj4_credits", "subj5_credits", "subj6_credits", "subj7_credits"
              ]

# Retrieve form data select name=, mentioned in name =

    Regn_No = request.form.get('regno')
    Name = request.form.get('sname')
    Semester = request.form.get('semester')

 # --- Subject 1 ---
    subj1_data = request.form.get("subj1")  # e.g., "Maths:::4"
    subj1_parts = subj1_data.split(":::")
    subj1 = subj1_parts[0]                  # Subject name
    subj1_credits = subj1_parts[1]          # Credits
    subj1_grade = request.form.get('grade1')

# --- Subject 2 ---
    subj2_data = request.form.get("subj2")  # e.g., "Physics:::2"
    subj2_parts = subj2_data.split(":::")
    subj2 = subj2_parts[0]                  # Subject name
    subj2_credits = subj2_parts[1]          # Credits
    subj2_grade = request.form.get('grade2')

# --- Subject 3 ---
    subj3_data = request.form.get("subj3")  # e.g., "Physics:::2"
    subj3_parts = subj3_data.split(":::")
    subj3 = subj3_parts[0]                  # Subject name
    subj3_credits = subj3_parts[1]          # Credits
    subj3_grade = request.form.get('grade3')

# --- Subject 4 ---
    subj4_data = request.form.get("subj4")  # e.g., "Physics:::2"
    subj4_parts = subj4_data.split(":::")
    subj4 = subj4_parts[0]                  # Subject name
    subj4_credits = subj4_parts[1]          # Credits
    subj4_grade = request.form.get('grade4')

# --- Subject 5 ---
    subj5_data = request.form.get("subj5")  # e.g., "Physics:::2"
    subj5_parts = subj5_data.split(":::")
    subj5 = subj5_parts[0]                  # Subject name
    subj5_credits = subj5_parts[1]          # Credits
    subj5_grade = request.form.get('grade5')

# --- Subject 6 ---
    subj6_data = request.form.get("subj6")  # e.g., "Physics:::2"
    subj6_parts = subj6_data.split(":::")
    subj6 = subj6_parts[0]                  # Subject name
    subj6_credits = subj6_parts[1]          # Credits
    subj6_grade = request.form.get('grade6')

# --- Subject 7 ---
    subj7_data = request.form.get("subj7")  # e.g., "Physics:::2"
    subj7_parts = subj7_data.split(":::")
    subj7 = subj7_parts[0]                  # Subject name
    subj7_credits = subj7_parts[1]          # Credits
    subj7_grade = request.form.get('grade7')

# Overwrite the CSV file each time

# # Save your data as csv file 'Single_Air_Quality.csv'

    with open(file_path, 'w', newline='') as f:
         writer = csv.writer(f)
         writer.writerow(headers)
         writer.writerow([Regn_No, Name,Semester,subj1,subj2,subj3,subj4,subj5,
             subj6,subj7, subj1_grade, subj2_grade, subj3_grade, subj4_grade,
             subj5_grade,subj6_grade,subj7_grade,subj1_credits,subj2_credits,
             subj3_credits,subj4_credits,subj5_credits,subj6_credits,
             subj7_credits])

# Cross verify
         print('Data has been written to', file_path)

    session['save_flag'] = True
# Redirect to the prediction page after saving the details

    return "<h2>Details Saved Successfully! Go back to the <a href='/'>Main Page</a>.</h2>"

@app.route('/predict')
def predict():

# Check if the save_flag is present in the session

  if not session.get('save_flag', False):
      # Display an alert box if details are not saved
      return '''
          <script>
              alert('Please enter User Details First, then click Predict Button.');
              window.location.href = '/';
          </script>
      '''

## ADD PREDICTION RELATED PROCESSING HERE

# # Read Single Input Data for prediction

  single_df_from_csv = pd.read_csv('/content/drive/MyDrive/Single_Univ_Data.csv')

# List of columns to drop when Semester == 4

  columns_to_drop = ['subj4', 'subj5', 'subj6', 'subj7', 'subj4_grade',
                     'subj5_grade', 'subj6_grade', 'subj7_grade', 'subj4_credits',
                     'subj5_credits', 'subj6_credits', 'subj7_credits']

## Drop the specified cols

  if (single_df_from_csv['Semester'] == 4).any():
    single_df_from_csv = single_df_from_csv.drop(columns=columns_to_drop)

## Drop NULL VALUES

  single_df_from_csv = single_df_from_csv.dropna()

  single_df_from_csv_copy = single_df_from_csv.copy()

# ### BLOCKED
# Example of encoding grades

  grade_mapping = {'S': 10, 'A+': 9, 'A': 8.5, 'B+': 8, 'B': 7.5, 'C+': 7, \
                   'C': 6.5, 'D': 6, 'P': 5.5, 'F': 0}

# # Convert alpha grade columns to numeric using the mapping

  if (single_df_from_csv['Semester'] != 4).any():
# Apply mapping for all subjects when Semester is NOT 4
      single_df_from_csv['subj1_grade'] = single_df_from_csv['subj1_grade'].map(grade_mapping)
      single_df_from_csv['subj2_grade'] = single_df_from_csv['subj2_grade'].map(grade_mapping)
      single_df_from_csv['subj3_grade'] = single_df_from_csv['subj3_grade'].map(grade_mapping)
      single_df_from_csv['subj4_grade'] = single_df_from_csv['subj4_grade'].map(grade_mapping)
      single_df_from_csv['subj5_grade'] = single_df_from_csv['subj5_grade'].map(grade_mapping)
      single_df_from_csv['subj6_grade'] = single_df_from_csv['subj6_grade'].map(grade_mapping)
      single_df_from_csv['subj7_grade'] = single_df_from_csv['subj7_grade'].map(grade_mapping)
  else:
# Apply mapping for only 3 subjects when Semester is 4
      single_df_from_csv['subj1_grade'] = single_df_from_csv['subj1_grade'].map(grade_mapping)
      single_df_from_csv['subj2_grade'] = single_df_from_csv['subj2_grade'].map(grade_mapping)
      single_df_from_csv['subj3_grade'] = single_df_from_csv['subj3_grade'].map(grade_mapping)

# ## ADD FAS COL HERE

# List of subject columns

  subject_cols_7 = ['subj1_grade', 'subj2_grade', 'subj3_grade', 'subj4_grade', \
                    'subj5_grade', 'subj6_grade', 'subj7_grade']

  subject_cols_3 = ['subj1_grade', 'subj2_grade', 'subj3_grade']

# # Create 'FAS' column with condition (1 if any subject equals 0, else 0), 7 & 3 subjs
# # Apply condition based on 'Semester'

  if (single_df_from_csv['Semester'] != 4).any():
      single_df_from_csv['FAS'] = (single_df_from_csv[subject_cols_7] == 0).any(axis=1).astype(int)
  else:
      single_df_from_csv['FAS'] = (single_df_from_csv[subject_cols_3] == 0).any(axis=1).astype(int)

## Filter relevant features

  if single_df_from_csv['Semester'].iloc[0] != 4:  # Check the first row
      single_df_selected_7 = single_df_from_csv[[
          "subj1_grade", "subj2_grade", "subj3_grade", "subj4_grade",
          "subj5_grade", "subj6_grade", "subj7_grade", "FAS",
          "subj1_credits", "subj2_credits", "subj3_credits", "subj4_credits",
          "subj5_credits", "subj6_credits", "subj7_credits"
      ]]
  else:
      single_df_selected_3 = single_df_from_csv[[
          "subj1_grade", "subj2_grade", "subj3_grade",
          "FAS", "subj1_credits", "subj2_credits", "subj3_credits"
      ]]

# Load the saved model for predictions - SEM1,SEM2,SEM3

  loaded_model_7 = tf.keras.models \
    .load_model('/content/drive/MyDrive/model_univ_7Subjs.keras', compile=False)

# Load the saved model for predictions - SEM4 Only

  loaded_model_3 = tf.keras.models \
    .load_model('/content/drive/MyDrive/model_univ_3Subjs.keras', compile=False)

## Predict using the appropriate saved models

# # # Replace small decimal predictions with 0.0
# # # Step 2: Apply the lambda function to replace values <= 0.1 with 0.0 $$

  if (single_df_from_csv['Semester'] != 4).any():
      predicted_single_7 = loaded_model_7.predict(single_df_selected_7)
      predicted_single_7[predicted_single_7 <= 0.1] = 0.0
      predicted_single_7 = np.round(predicted_single_7, 1)
      single_features_7 = single_df_from_csv_copy.copy()
      single_features_7["Predicted_SGPA"] = predicted_single_7
  else:
      predicted_single_3 = loaded_model_3.predict(single_df_selected_3)
      predicted_single_3[predicted_single_3 <= 0.1] = 0.0
      predicted_single_3 = np.round(predicted_single_3, 1)
      single_features_3 = single_df_from_csv_copy.copy()
      single_features_3["Predicted_SGPA"] = predicted_single_3

## Attach Final Alpha Grade using Predicted_SGPA

  if single_df_from_csv['Semester'].iloc[0] != 4:  # Check the first row
      single_features_7['Predicted_SGPA'] = single_features_7['Predicted_SGPA'].astype(float)
  else:
      single_features_3['Predicted_SGPA'] = single_features_3['Predicted_SGPA'].astype(float)

# Function to map Predicted SGPA Back to Alpha Grade using simple if-elif conditions

  def get_grade(sgpa):
      if sgpa >= 9.0:
          return 'S'
      elif sgpa >= 8.5:
          return 'A+'
      elif sgpa >= 8.0:
          return 'A'
      elif sgpa >= 7.5:
          return 'B+'
      elif sgpa >= 7.0:
          return 'B'
      elif sgpa >= 6.5:
          return 'C+'
      elif sgpa >= 6.0:
          return 'C'
      elif sgpa >= 5.5:
          return 'D'
      elif sgpa >= 5.0:
          return 'P'
      else:
          return 'F'

# # Apply the above function to the DataFrame

  if single_df_from_csv['Semester'].iloc[0] != 4:  # Check the first row
      single_features_7['Predicted_Grade'] = single_features_7['Predicted_SGPA'].apply(get_grade)
  else:
      single_features_3['Predicted_Grade'] = single_features_3['Predicted_SGPA'].apply(get_grade)

## Select relevant cols for mark list

  if single_df_from_csv['Semester'].iloc[0] != 4:  # Check the first row
      mark_list_cols = ["Regn_No", "Name", "Semester", "subj1_grade",
                        "subj2_grade", "subj3_grade", "subj4_grade",
                        "subj5_grade", "subj6_grade", "subj7_grade",
                        "Predicted_SGPA", "Predicted_Grade", "subj1",
                        "subj2", "subj3", "subj4", "subj5", "subj6", "subj7"]
      mark_list_df = single_features_7[mark_list_cols]  # Use _7
  else:
      mark_list_cols = ["Regn_No", "Name", "Semester", "subj1_grade",
                        "subj2_grade", "subj3_grade",
                        "Predicted_SGPA", "Predicted_Grade", "subj1",
                        "subj2", "subj3"]
      mark_list_df = single_features_3[mark_list_cols]  # Use _3

# Sample DataFrame (assuming mark_list_df is already created)

  student_info_cols = ["Regn_No", "Name", "Semester", "Predicted_SGPA", "Predicted_Grade"]

  if single_df_from_csv['Semester'].iloc[0] != 4:  # Semester is NOT 4 (Use _7)
      subject_cols = [f"subj{i}" for i in range(1, 8)]
      grade_cols = [f"subj{i}_grade" for i in range(1, 8)]
  else:  # Semester is 4 (Use _3)
      subject_cols = [f"subj{i}" for i in range(1, 4)]
      grade_cols = [f"subj{i}_grade" for i in range(1, 4)]

# Convert student details into a vertical format

  student_info_melted = mark_list_df.melt(
      value_vars=student_info_cols, var_name="Column", value_name="Data"
  )

# Create subject-grade pairs as a structured DataFrame

  subject_grade_df = pd.DataFrame({
      "Column": [mark_list_df.iloc[0][subj] \
        for subj in subject_cols if subj in mark_list_df.columns],
      "Data": [mark_list_df.iloc[0][grade] \
          for grade in grade_cols if grade in mark_list_df.columns]
      })

# Concatenate student info + subjects

  final_df = pd.concat([student_info_melted, subject_grade_df], ignore_index=True)

## Re-Arrange Rows in final_df, that is move SGPA and Predicted_Grade
## to last rows

# 1. Keep only the rows that are NOT 'SGPA' or 'Predicted_Grade'

  remaining_rows = final_df.loc[final_df['Column'] != 'Predicted_SGPA']
  remaining_rows = remaining_rows.loc[remaining_rows['Column'] != 'Predicted_Grade']

# 2. Select the rows that ARE 'SGPA' or 'Predicted_Grade'

  rows_to_move = final_df.loc[(final_df['Column'] == 'Predicted_SGPA') | \
                (final_df['Column'] == 'Predicted_Grade')]

# 3. Concatenate them

  final_df = pd.concat([remaining_rows, rows_to_move], ignore_index=True)

# ## Save final_df in append mode not needed

## Remove extra zeores in predicted_sgpa when displayed

  columns_to_extract_7 = ['Column', 'Data']

  styled_df_7 = final_df.style.map(
        lambda x: 'background-color: aliceblue;',
        subset=columns_to_extract_7
    ).format(
        lambda x: '{:.1f}'.format(float(x))
        if '.' in str(x) and str(x).replace('.', '', 1).isdigit() and (final_df['Column'] == 'Predicted_SGPA').any()
        else '{:.0f}'.format(float(x))
        if str(x).isdigit() or str(x).replace('.', '', 1).isdigit()
        else x,
        subset=['Data']
    )

  html_table_7 = styled_df_7.hide(axis="index").set_table_styles([
        {'selector': 'thead', 'props': [('visibility', 'hidden')]},
        {'selector': 'th', 'props': [('color', 'red'), ('padding', '5px'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [
            ('vertical-align', 'middle'),
            ('padding', '5px'),
            ('font-size', '14px'),
            ('color', 'green'),
            ('font-weight', 'bold'),
            ('text-align', 'center'),
            ('font-family', 'Times New Roman'),
            ('white-space', 'nowrap'),
            ('width', 'auto')
        ]}
    ]).to_html()

  output_frame_7 = f"""
        <div style="border: 2px solid black; padding: 10px; width: 75%;
            background-color: lightgrey; height: auto;">
            <h3 style='color:blue; text-align:center;'>KVM College of Engineering & IT</h3>
            {html_table_7}
        </div>
      """

# ## PRINT OUT

  html_path = "/content/drive/MyDrive/marklist_output.html"
  pdf_path = "/content/drive/MyDrive/marklist_output.pdf"

  html_rendered = render_template_string("""
      <!DOCTYPE html>
      <html>
      <head>
          <title>Result</title>
          <style>
              .btn {
                  font-size: 20px;
                  padding: 10px 20px;
                  margin: 15px 15px 0 0;
                  color: white;
                  border: none;
                  border-radius: 8px;
                  text-decoration: none;
                  display: inline-block;
              }
              .download-btn {
                  background-color: #007BFF;
              }
              .goback-btn {
                  background-color: #28a745;
              }
              /* Hide elements with .no-print class in PDF/print view */
              @media print {
                  .no-print {
                      display: none;
                  }
              }
             @page {
                  size: A4 landscape;  /* or: auto */
                  margin: 10mm;
              }
              body {
                  width: 100%;
                  overflow-x: auto;
              }
              table {
                  width: 100%;
                  table-layout: auto;
                  border-collapse: collapse;
              }
              td, th {
                  word-wrap: break-word;
                  white-space: nowrap;
              }

          </style>
      </head>
      <body>
          {{ table_html | safe }}
          <br><br>
          <a href="/download" class="btn download-btn no-print">Download as PDF</a>
          <a href="/" class="btn goback-btn no-print">Go Back</a>
      </body>
      </html>
  """, table_html=output_frame_7)

# ✅ Save to HTML file

  with open("/content/drive/MyDrive/marklist_output.html", "w") as f:
      f.write(html_rendered)

  # ✅ Return the rendered HTML

  return html_rendered

@app.route('/download')
def download_pdf():

# Convert HTML to PDF
    html_file = "/content/drive/MyDrive/marklist_output.html"
    pdf_output = "/content/drive/MyDrive/marklist_output.pdf"

# Generate PDF from HTML using WeasyPrint
    weasyprint.HTML(html_file).write_pdf(pdf_output)

# Send the generated PDF for download
    return send_file(pdf_output, as_attachment=True)

@app.route('/accuracyDetails')
def accuracyDetails():

# # Accuracy results for Regression Models

    accuracy_rsquare = 99.987   # Over all Accurcay
    accuracy_maep = 0.00241     # Less than 1 is excellent
    accuracy_mae = 0.0188       # Very low and negligible
    accuracy_mse = 0.0019       # Very low and negligible

    return f"""
    <html>
    <head>
        <style>
            .abstract {{
                margin: 20px auto;
                max-width: 900px;
                padding: 15px;
                background: #FFFFE0; /* Light yellow background */
                color: #155724;
                font-size: 18px;
                font-weight: bold;
                box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
                border-radius: 5px;
                border: 1px solid #c3e6cb;
            }}
            .buttons {{
                margin-top: 20px;
               text-align: center;
            }}
        </style>
    </head>
    <body>
        <h2 style="color: magenta; text-align: center;">Accuracy of this Model is shown below</h2>
        <br>
        <div class="abstract">
            <h3 style="color: green;">Over all Accuracy % [R-Square Score] = {accuracy_rsquare}%</h3>
            <h3 style="color: green;">Mean Squared Error Percent[Less than 1 Excellent]] = {accuracy_maep}%</h3>
            <h3 style="color: green;">Mean Absoulte Error[Very low-Excellent] = {accuracy_mae}</h3>
            <h3 style="color: green;">Mean Squared Error[Very low-Excellent] = {accuracy_mse}</h3>
        </div>
        <div class="buttons">
            <br><h2><a href="/">Go back to the Main Page</a></h2>
        </div>
    </body>
    </html>
    """

@app.route('/AboutThis')
def AboutThis():

    return f"""
    <html>
    <head>
        <style>
            .abstract {{
                margin: 20px auto;
                max-width: 900px;
                padding: 15px;
                background: #FFFFE0; /* Light yellow background */
                color: #155724;
                font-size: 20px;
                font-weight: bold;
                box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
                border-radius: 5px;
                border: 1px solid #c3e6cb;
            }}
            .buttons {{
                margin-top: 20px;
                text-align: center;
            }}
        </style>
    </head>
    <body>
        <h2 style="color: magenta; text-align: center;">Details of this Project is shown below</h2>
        <br>
        <div class="abstract";>
        <h2>Objective</h2>
        <p>
            In Kerala Technological University, semester-wise mark lists provide only alphabetic grades \
            for each subject, without displaying the Semester Grade Point Average (SGPA) or the final grade. \
            As a result, students are required to manually compute their SGPA by converting the alphabetic \
            grades into numerical grade points, multiplying them by corresponding credit values, and calculating \
            the weighted average. This process is not only time-consuming but also prone to human error. To overcome \
            this challenge, this project proposes the use of Deep Neural Networks (DNN) to accurately and efficiently \
            predict a student's SGPA and final grade based on their subject-wise alphabetic grades. This automated \
            approach ensures both speed and accuracy, eliminating the potential for manual calculation errors.
        </p>
        <h2>Motivation</h2>
        <p>
            Currently, there are no readily available or customized models on the World Wide Web that address the \
            challenge of automatically calculating SGPA and final grades based on subject-wise alphabetic grades. \
            This gap presents a significant opportunity for innovation. The proposed Deep Neural Network (DNN) \
            model is designed to bridge this gap by allowing students to input their subject names along with \
            the corresponding alphabetic grades. The model then processes this data to instantly predict the \
            SGPA and final grade with high accuracy. As an added convenience, the final output is generated \
            in a well-structured PDF format, providing students with a clear and error-free summary of their \
            academic performance within seconds.
        </p>
        <h2 style="color: red;">Core Algorithms used:-</h2>
        <p>
            Tensorflow and Keras are the core DNN algorithms. The Neuron Distribution is as \
            follows: 512, 256, 128, 64, 32, 16. Total weights of these Neurons or Hidden \
            layers are, 1,75,088. These weights are capable of giving accurate predictions.
        </p>

        </div>
        <div class="buttons">
            <br><h2><a href="/">Go back to the Main Page</a></h2>
        </div>
    </body>
    </html>
    """

# Start Flask app

def run_flask():
    app.run(port=5000, debug = False)

# Set up ngrok tunnel and extract the public URL string with
# host header to skip warning

def setup_ngrok():

# Start ngrok after the Flask app is initialized

    time.sleep(2)  # Sleep for a moment to ensure Flask is running before ngrok starts

    public_url = ngrok.connect("5000", host_header="ngrok-skip-browser-warning").public_url

# Make the public URL clickable

    display(HTML(f"<h3 style='color: red;'>Click here to open your web application: \
      <a href='{public_url}' target='_blank'>{public_url}</a></h3>"))

    return public_url

# Run the Flask app in a separate thread

thread = threading.Thread(target=run_flask)
thread.start()

# Set up ngrok tunnel in the main thread to ensure it's ready after Flask starts

public_url = setup_ngrok()